# 01. Overview

#### 가설
* 가설1 : 거주 인구가 많은 지역에 스타벅스 매장이 많이 입지해 있을 것이다.
* 가설2 : 직장인이 많은 지역에 스타벅스 매장이 많이 입지해 있을 것이다.
* 가설3 : 유동인구가 많은 지역에 스타벅스 매장이 많이 입지해 있을 것이다.
* 가설4 : 아 집가고싶다.
* 가설5 : 역 근처에 스타벅스 매장이 많이 입지해 있을 것이다.
* 가설6 : 주꾸미 볶음 먹고싶다.
* 가설7 : 양꼬치 먹고싶다.
* 가설8 : 지대가 비싼 지역에 스타벅스 매장이 많이 입지해 있을 것이다.
* 가설9 : 대학교 근처에 스타벅스 매장이 많이 입지해 있을 것이다.
* 가설10 : 스타벅스 커피는 진짜 개맛없다.
* 가설11 : 인근에 카페가 많은 지역에 스타벅스 매장이 많이 입지해 있을 것이다.
* 가설12 : 경쟁 커피 프랜차이즈 점포가 있다면 스타벅스 매장이 근처에 위치해 있을 것이다.

* 스타벅스 입지전략을 분석 : 가설 1, 2에 대한 검증 진행 // 돌돌1,2

#### 데이터 수집

* 1. 스타벅스 사이트에서 서울시에 존재하는 매장의 정보 수집
    * 매장명, 위도, 경도, 매장타입, 주소, 전화번호
    * https://www.starbucks.co.kr/
* 2. 서울시 주민등록 인구 (구별) 통계 정보 수집
    * 서울시 열린데이터 광장
        * https://data.seoul.go.kr/dataList/419/S/2/datasetView.do
* 3. 서울시 사업체 현황(산업 대분류별 / 동별) 통계 정보
    * 서울시 열린데이터 광장
        * https://data.seoul.go.kr/dataList/104/S/2/datasetView.do

## 02. 데이터 수집

# 02-1 서울시 스타벅스 매장 목록 데이터 수집 : 크롤링


    * https://www.istarbucks.co.kr

In [85]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service

service = Service()


options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service, options=options)



In [86]:
url = 'https://www.starbucks.co.kr/store/store_map.do?disp=locale'

driver.get(url)

In [87]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC 

seoul_btn = '#container > div > form > fieldset > div > section > article.find_store_cont > article > article:nth-child(4) > div.loca_step1 > div.loca_step1_cont > ul > li:nth-child(1) > a'
try : WebDriverWait(driver,20).until(EC.element_to_be_clickable\
                               ((By.CSS_SELECTOR, seoul_btn))).click() # 20초 내로 실행되지 않는다면 TimeoutError 
    
except : 
    print('해당 페이지가 20초 내로 열리지 않았음')

In [88]:
# '전체' 버튼 요소를 찾아 클릭

all_btn = '#mCSB_2_container > ul > li:nth-child(1) > a'

try : WebDriverWait(driver,20).until(EC.element_to_be_clickable\
                               ((By.CSS_SELECTOR, all_btn))).click() # 20초 내로 실행되지 않는다면 TimeoutError 
    
except : 
    print('해당 페이지가 20초 내로 열리지 않았음')

In [93]:
html = driver.page_source

In [94]:
# BeautifulSoup로 파서 생성

soup = bs(html, 'html.parser')

starbucks_soup_list = soup.select('li.quickResultLstCon')
print(len(starbucks_soup_list))

611


In [96]:
starbucks_soup_list[10]

<li class="quickResultLstCon" data-code="3588" data-hlytag="null" data-index="10" data-lat="37.488008" data-long="127.102650" data-name="수서역R" data-storecd="1389" style="background:#fff"> <strong data-my_siren_order_store_yn="N" data-name="수서역R" data-store="1389" data-yn="N">수서역R  </strong> <p class="result_details">서울특별시 강남구 광평로 281 (수서동)<br/>1522-3232</p> <i class="pin_reserve">리저브 매장 2번</i></li>

In [97]:
starbucks_soup_list[0].find('i')

<i class="pin_general">리저브 매장 2번</i>

In [98]:
name = [] # 매장명
lat = [] # 위도
lng = [] # 경도
store_type = [] # 매장 타입
address = [] # 주소
tel = [] # 전화번호

for i in range(len(starbucks_soup_list)):
    name.append(starbucks_soup_list[i]['data-name'].strip())
    lat.append(starbucks_soup_list[i]['data-lat'].strip())
    lng.append(starbucks_soup_list[i]['data-long'].strip())
    try:
        store_type.append(starbucks_soup_list[i].find('i').text) # genenral 말고 reserve 도 있어서 그걸 못 찾아서 none으로 불러와졌나봐 657

        # store_type.append(starbucks_soup_list[i].select('i')[0]['class'][0])

    except:
        store_type.append('일반매장')
    
    
    # add_tel = str(starbucks_soup_list[i].select('p.result_details')[0]).split('<br/>')

    # address = add_tel[0].split('>')[1]
    # tel = add_tel[1].split('<')[0]


    address.append(starbucks_soup_list[i].find('p', {'class' : 'result_details'}).contents[0].strip())
    tel.append(starbucks_soup_list[i].find('p', {'class' : 'result_details'}).contents[-1].strip())


In [112]:
df = pd.DataFrame({'매장명' : name, '위도' : lat, '경도' : lng, '매장타입' : store_type, '주소' : address, '전화번호' : tel})

In [170]:
df.to_csv('./files/starbucks.csv')

# 2. 서울시 주민등록 인구 (구별) 통계 정보 수집
    * 서울시 열린데이터 광장
        * https://data.seoul.go.kr/dataList/419/S/2/datasetView.do

In [131]:
sgg_pop_df = pd.read_csv('./files/report.txt', sep = '\t', header = 1)


In [132]:
columns = {
    '시점': 'GIGAN',
    '동별(1)': 'SUM',
    '동별(2)': 'JACHIGU',
    '소계': 'SEDAE',
    '소계.1': 'GYE_1',
    '남자': 'NAMJA_1',
    '여자': 'YEOJA_1',
    '소계.2': 'GYE_2', # 한국인
    '남자.1': 'NAMJA_2',
    '여자.1': 'YEOJA_2',
    '소계.3': 'GYE_3', # 등록외국인
    '남자.2': 'NAMJA_3',
    '여자.2': 'YEOJA_3',
    '소계.4': 'SEDAEDANGINGU',
    '소계.5': 'N_65SEISANGGORYEONGJA'
}
sgg_pop_df.rename(columns = columns, inplace = True)


In [133]:
sgg_pop_df

,GIGAN,SUM,JACHIGU,SEDAE,GYE_1,NAMJA_1,YEOJA_1,GYE_2,NAMJA_2,YEOJA_2,GYE_3,NAMJA_3,YEOJA_3,SEDAEDANGINGU,N_65SEISANGGORYEONGJA
0,2023 1/4,합계,소계,4463385,9668008,4672170,4995838,9426404,4566299,4860105,241604,105871,135733,2.11,1690961
1,2023 1/4,합계,종로구,72679,152212,72819,79393,141060,68170,72890,11152,4649,6503,1.94,28265
2,2023 1/4,합계,중구,63862,131390,63576,67814,120963,58699,62264,10427,4877,5550,1.89,25353
3,2023 1/4,합계,용산구,109735,232482,112608,119874,217756,104640,113116,14726,7968,6758,1.98,39478
4,2023 1/4,합계,성동구,133513,287240,139171,148069,280240,136233,144007,7000,2938,4062,2.10,48238
5,2023 1/4,합계,광진구,169787,350925,167983,182942,336801,162209,174592,14124,5774,8350,1.98,54854
6,2023 1/4,합계,동대문구,171140,354884,172178,182706,337574,165933,171641,17310,6245,11065,1.97,65154
7,2023 1/4,합계,중랑구,187898,389928,191122,198806,385003,189186,195817,4925,1936,2989,2.05,76116
8,2023 1/4,합계,성북구,197859,441855,210937,230918,430100,206634,223466,11755,4303,7452,2.17,78427
9,2023 1/4,합계,강북구,144557,296934,143493,153441,292762,142028,150734,4172,1465,2707,2.03,67056


In [137]:
# 내가 한 필요없는 데이터 떨구기

# sgg_pop_df.drop(0, axis= 0, inplace= True)

KeyError: '[0] not found in axis'

In [138]:
# 위 방법대로 없애는 것 말고 아래처럼 filter 씌워서 해도 됨

conditions = sgg_pop_df['JACHIGU'] != '소계'

In [140]:
sgg_pop_df_selected = sgg_pop_df[conditions]

In [142]:
sgg_pop_df_selected.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 1 to 25
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   GIGAN                  25 non-null     object 
 1   SUM                    25 non-null     object 
 2   JACHIGU                25 non-null     object 
 3   SEDAE                  25 non-null     int64  
 4   GYE_1                  25 non-null     int64  
 5   NAMJA_1                25 non-null     int64  
 6   YEOJA_1                25 non-null     int64  
 7   GYE_2                  25 non-null     int64  
 8   NAMJA_2                25 non-null     int64  
 9   YEOJA_2                25 non-null     int64  
 10  GYE_3                  25 non-null     int64  
 11  NAMJA_3                25 non-null     int64  
 12  YEOJA_3                25 non-null     int64  
 13  SEDAEDANGINGU          25 non-null     float64
 14  N_65SEISANGGORYEONGJA  25 non-null     int64  
dtypes: float

In [143]:
# 분석에 필요한 column 선택

columns = ['JACHIGU', 'GYE_1']

sgg_pop_df_final = sgg_pop_df_selected[columns]

In [146]:
sgg_pop_df_final.columns = ['시군구명', '주민등록인구']

sgg_pop_df_final

,시군구명,주민등록인구
1,종로구,152212
2,중구,131390
3,용산구,232482
4,성동구,287240
5,광진구,350925
6,동대문구,354884
7,중랑구,389928
8,성북구,441855
9,강북구,296934
10,도봉구,312858


In [168]:
sgg_pop_df_final.to_excel('./files/sgg_pop.xlsx', index= False)

# 3. 서울시 사업체 현황(산업 대분류별 / 동별) 통계 정보
    * 서울시 열린데이터 광장
        * https://data.seoul.go.kr/dataList/104/S/2/datasetView.do

In [149]:
sgg_biz_df = pd.read_csv('./files/report1.txt', sep = '\t', header = 3)

sgg_biz_df

,시점,동별(1),동별(2),동별(3),소계,여성대표자,소계.1,남,여,소계.2,...,소계.30,소계.31,소계.32,소계.33,소계.34,소계.35,소계.36,소계.37,소계.38,소계.39
0,2021,합계,종로구,소계,48361,16563,275063,153210,121853,4,...,100,17436,1082,13298,830,19143,1141,5650,2591,7297
1,2021,합계,종로구,사직동,5035,1708,56013,28597,27416,1,...,28,7047,132,927,120,1207,114,1612,285,944
2,2021,합계,종로구,삼청동,956,423,4978,2482,2496,-,...,3,1264,20,287,9,40,37,307,39,146
3,2021,합계,종로구,부암동,894,416,3762,1665,2097,-,...,3,65,55,1338,30,286,39,99,81,226
4,2021,합계,종로구,평창동,1321,577,4321,2068,2253,-,...,3,115,70,448,40,521,81,195,123,281
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
446,2021,합계,강동구,천호2동,4863,1774,14840,7434,7406,-,...,3,100,127,561,164,1814,103,316,339,598
447,2021,합계,강동구,길동,5966,1989,23814,12083,11731,1,...,5,263,173,879,187,5278,157,357,443,895
448,2021,합계,강동구,강일동,1443,453,5960,3816,2144,-,...,2,63,117,712,75,739,35,99,80,117
449,2021,합계,강동구,상일1동,1679,666,7035,2959,4076,1,...,2,62,116,1169,44,2299,42,249,106,161


In [150]:
columns = {
    '시점': 'GIGAN',
    '동별(1)': 'GIGAN',
    '동별(2)': 'JACHIGU',
    '동별(3)': 'DONG',
    '소계': 'SAEOPCHESU_1',
    '여성대표자': 'YEOSEONGDAEPYOJA',
    '소계.1': 'GYE',
    '남': 'NAM',
    '여': 'YEO'
}
sgg_biz_df.rename(columns = columns, inplace = True)
sgg_biz_df.head(10)

,GIGAN,GIGAN,JACHIGU,DONG,SAEOPCHESU_1,YEOSEONGDAEPYOJA,GYE,NAM,YEO,소계.2,...,소계.30,소계.31,소계.32,소계.33,소계.34,소계.35,소계.36,소계.37,소계.38,소계.39
0,2021,합계,종로구,소계,48361,16563,275063,153210,121853,4,...,100,17436,1082,13298,830,19143,1141,5650,2591,7297
1,2021,합계,종로구,사직동,5035,1708,56013,28597,27416,1,...,28,7047,132,927,120,1207,114,1612,285,944
2,2021,합계,종로구,삼청동,956,423,4978,2482,2496,-,...,3,1264,20,287,9,40,37,307,39,146
3,2021,합계,종로구,부암동,894,416,3762,1665,2097,-,...,3,65,55,1338,30,286,39,99,81,226
4,2021,합계,종로구,평창동,1321,577,4321,2068,2253,-,...,3,115,70,448,40,521,81,195,123,281
5,2021,합계,종로구,무악동,761,176,1891,960,931,-,...,1,14,33,168,10,560,14,27,51,168
6,2021,합계,종로구,교남동,592,256,8371,4445,3926,-,...,2,37,32,191,30,3568,17,64,57,154
7,2021,합계,종로구,가회동,969,436,14180,10707,3473,-,...,3,528,39,726,19,88,47,132,69,178
8,2021,합계,종로구,종로1.2.3.4가동,17286,5412,107067,63437,43630,1,...,24,4654,339,2890,221,1850,243,1031,911,2509
9,2021,합계,종로구,종로5.6가동,7972,2614,22872,14187,8685,1,...,5,352,35,154,94,441,50,136,252,764


In [157]:
conditions = sgg_biz_df['DONG'] == '소계'

In [158]:
sgg_biz_df_select = sgg_biz_df[conditions]

In [159]:
sgg_biz_df_select

,GIGAN,GIGAN,JACHIGU,DONG,SAEOPCHESU_1,YEOSEONGDAEPYOJA,GYE,NAM,YEO,소계.2,...,소계.30,소계.31,소계.32,소계.33,소계.34,소계.35,소계.36,소계.37,소계.38,소계.39
0,2021,합계,종로구,소계,48361,16563,275063,153210,121853,4,...,100,17436,1082,13298,830,19143,1141,5650,2591,7297
18,2021,합계,중구,소계,70308,25794,386564,213782,172782,1,...,65,16141,644,9425,813,11585,632,3258,2175,7632
34,2021,합계,용산구,소계,29680,10454,152605,84314,68291,4,...,45,3416,844,7352,632,8219,651,3006,1866,5572
51,2021,합계,성동구,소계,41665,13447,203221,116978,86243,7,...,45,5243,1283,10453,815,9635,844,2956,2341,7334
69,2021,합계,광진구,소계,33706,12297,129707,71124,58583,3,...,38,3592,1467,14698,981,12785,1139,3202,2799,5121
85,2021,합계,동대문구,소계,42813,14480,146383,78105,68278,2,...,49,4023,1288,16173,1085,17183,889,2282,2790,4906
100,2021,합계,중랑구,소계,39310,13471,113144,59221,53923,-,...,47,4108,1151,7668,1111,14806,1029,2273,3099,4937
117,2021,합계,성북구,소계,34712,13393,124004,60275,63729,1,...,49,4607,1806,21226,1131,16668,1043,2444,2931,5502
138,2021,합계,강북구,소계,26385,9807,80222,38440,41782,1,...,42,3530,890,6164,867,11540,869,1885,2508,4270
152,2021,합계,도봉구,소계,25391,8886,79097,41600,37497,2,...,37,4256,1114,7191,859,12804,738,1594,2264,4184


In [163]:
# 분석에 필요한 colum 선택

sgg_biz_df_final = sgg_biz_df_select[['JACHIGU', 'GYE' , 'SAEOPCHESU_1']]
sgg_biz_df_final.columns = ['시군구명', '종사자수', '사업체수']

In [164]:
sgg_biz_df_final

,시군구명,종사자수,사업체수
0,종로구,275063,48361
18,중구,386564,70308
34,용산구,152605,29680
51,성동구,203221,41665
69,광진구,129707,33706
85,동대문구,146383,42813
100,중랑구,113144,39310
117,성북구,124004,34712
138,강북구,80222,26385
152,도봉구,79097,25391


In [165]:
sgg_biz_df_final.reset_index(drop= True)

,시군구명,종사자수,사업체수
0,종로구,275063,48361
1,중구,386564,70308
2,용산구,152605,29680
3,성동구,203221,41665
4,광진구,129707,33706
5,동대문구,146383,42813
6,중랑구,113144,39310
7,성북구,124004,34712
8,강북구,80222,26385
9,도봉구,79097,25391


In [167]:
sgg_biz_df_final.to_excel('./files/sgg_biz.xlsx', index = False)

'''
[file list]

1. Data collection
- starbucks.csv : 스타벅스 정보(크롤링)
- seoul_sgg_list.xlsx : 서울시 시군구 정보(공공데이터)
- report.txt : 시군구별 인구수 현황(공공데이터)
- report1.txt : 시군구별 사업체 현황(공공데이터)


2. Data processing
- sgg_pop.xlsx : 시군구별 인구수 현황(가공)
- sgg_biz.xlsx : 시군구별 사업체 현황(가공)

3. 

'''
